In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization,MaxPooling2D,Flatten,Dropout


In [2]:

cifar10=tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train,x_test=x_train/255,x_test/255
print(x_train.shape)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)


In [3]:
#the data is 2d
#convolution expecst height x width x color
y_train=y_train.flatten()
y_test=y_test.flatten()
print(y_train.shape)

(50000,)


In [4]:

k=len(set(y_train))
print(k)

10


In [7]:

#build the model using functionaln api
i=tf.keras.layers.Input(shape=x_train[0].shape)

x=tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(i)
x=BatchNormalization()(x)
x=tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(x)
x=BatchNormalization()(x)
x=MaxPooling2D((2,2))(x)

x=tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(i)
x=BatchNormalization()(x)
x=tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
x=BatchNormalization()(x)
x=MaxPooling2D((2,2))(x)

x=tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(i)
x=BatchNormalization()(x)
x=tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
x=BatchNormalization()(x)
x=MaxPooling2D((2,2))(x)

x=Flatten()(x)
x=Dropout(0.2)(x)

x=Dense(1024,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(k,activation='relu')(x)
model=tf.keras.Model(i,x)

In [13]:
from keras.engine.training import optimizer
#compile and fit
#note make sure you are using GPU for this
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=7)

In [ ]:
#fit with data augmentation

#fit with data augmentation
#if you run this after calling model.fit it will continue from where it left training
batch_size=32
data_generator=tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,
                                                               rotation_range=0.2,horizontal_flip=False,vertical_flip=False)
train_generator=data_generator.flow(x_train,y_train,batch_size)
steps_per_epochs=x_train.shape[0//batch_size]
r=model.fit_generator(train_generator,validation_data=(x_test,y_test),steps_per_epoch=steps_per_epochs,epochs=10)

In [15]:

plt.plot(r.history['accuracy'],label="accuracy")
plt.plot(r.history['val_accuracy'],label='val_accuracy')

In [15]:

plt.plot(r.history['loss'],label="loss")
plt.plot(r.history['val_loss'],label='val_loss')


In [15]:

#show some misclassified example
misclassified_idx=np.where(p_test!=y_test)[0]
i=np.random.choice(misclassified_idx)
plt.imshow(x_test[i],cmap='gray')
plt.title("true label: %s predicted: %s"%(y_test[i],p_test[i]))
     